In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0s/step


In [9]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [10]:
train.shape

(120, 4)

In [5]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [6]:
def input_fn(features, labels, training=True, batch_size=256):
  # Pretvori is pandas u tf DataSet
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #Uradi shufle N puta
  if training:
    dataset = dataset.shuffle(1000).repeat()
  
  return dataset.batch(batch_size) #I odredjenji batch size

In [11]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))


In [12]:
classifier = tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\TEODOR~1\\AppData\\Local\\Temp\\tmp04gchoep', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [13]:
#Kao u Exercise 2 morali smo da imamo funkciju koja vraca funkciju kao odgovor, to smo u 2. uradili u input function
#Dok ovde smo za input function uradili tek posle prilikom trening sa LAMBDA  izrazom, zasto je ovo ovako mora da se proveri
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True), steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\TEODOR~1\AppData\Local\Temp\tmp04gchoep\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.2787906, step = 0
INFO:tensorflow:global_step/sec: 411.948
INFO:tensorflow:loss = 0.97209954, step = 100 (0.243 sec)
INFO:tensorflow:global_step/sec: 566.467
INFO:tensorflow:loss = 0.9069021, step = 200 (0.178 sec

In [14]:
#Sada treba evaluirati nas trening  i traning=False jer netreba shuffle podataka jer je samo evaluacija
classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-21T12:48:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\TEODOR~1\AppData\Local\Temp\tmp04gchoep\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.29125s
INFO:tensorflow:Finished evaluation at 2021-11-21-12:48:08
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.93333334, average_loss = 0.41588396, global_step = 5000, loss = 0.41588396
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\TEODOR~1\AppData\Local\Temp\tmp04gchoep\model.ckpt-5000


{'accuracy': 0.93333334,
 'average_loss': 0.41588396,
 'loss': 0.41588396,
 'global_step': 5000}